# **Analytics 2 :** <font color=#DF4807>**Transformers**</font>

In [2]:
!pip install datasets --quiet
import pandas as pd
from datasets import load_dataset

c:\Users\Poorn\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
emotions = load_dataset("SetFit/emotion")
emotions_train = emotions['train']

Using the latest cached version of the dataset since SetFit/emotion couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\Poorn\.cache\huggingface\datasets\SetFit___emotion\default\0.0.0\6c362e04d016f6b6a9377e85c3b944140f0b96c9 (last modified on Fri May  3 13:05:01 2024).


In [4]:
# emotions.set_format(type='pandas')
# df = emotions['train'][:]
# df.head(10)

## **Tasks**

1. Fine tune a classification model using the dataset provided.
2. Test your trained model on unseen data.
3. If you are having difficulties with the multiclass classification, you can tweak the labels into 2 i.e. positive or negative to simplify.

In [56]:
pip install  -q scikit-learn 


Note: you may need to restart the kernel to use updated packages.


In [57]:
#install requirements

!pip install transformers datasets torch  -q gwpy
!pip install sentencepiece -q gwpy

from datasets import load_dataset
from datasets import load_metric #for access to metrics like F1 score, precision, recall...

from transformers import pipeline
from transformers import DataCollatorWithPadding #help batch data and pad them to a common lenght

from transformers import AutoTokenizer #generic tokenizer that we can set to match a pre-trained model
from transformers import AutoModelForQuestionAnswering, DistilBertConfig #AutoModel class plus bert config
from transformers import Trainer, TrainingArguments #used for fine tuning a model

from sklearn.metrics import f1_score
import torch

In [58]:

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

c:\Users\Poorn\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [60]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

In [61]:
tokenized_emotions = emotions.map(preprocess_function, batched=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

Map: 100%|██████████| 16000/16000 [00:00<00:00, 77974.08 examples/s]

Map: 100%|██████████| 2000/2000 [00:00<00:00, 71367.51 examples/s]

Map: 100%|██████████| 2000/2000 [00:00<00:00, 74109.55 examples/s]


In [62]:
tokenized_emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [63]:
# tokenized_test = emotions['test'].map(preprocess_function, batched=True)

# tokenized_train = emotions['train'].map(preprocess_function, batched=True)

In [64]:
# tokenized_train

In [65]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,padding=True)

In [66]:
id2label = {0: "sadness",1:'joy', 2: "love",3:'anger',4:'fear',5:'surprise'}

label2id = {"sadness": 0, "joy": 1,"love":2,"anger":3,"fear":4,"surprise":5}

In [67]:
from transformers import AutoModelForSequenceClassification #automodel class provided by HF, allows us to select models for classification tasks
#NOTE: sequence classification tasks include sentiment analysis, text categorization, and natural language inference.
from transformers import TrainingArguments, Trainer #classes that we need for fine tuning

In [68]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [69]:

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=6,id2label=id2label,label2id=label2id,ignore_mismatched_sizes=True)

c:\Users\Poorn\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
pip install accelerate -U --quiet

Note: you may need to restart the kernel to use updated packages.


In [71]:
pip install transformers[torch] --quiet


Note: you may need to restart the kernel to use updated packages.


In [72]:
import os
os.environ['PYDEVD_USE_FRAME_EVAL'] = 'NO'


In [73]:
training_args = TrainingArguments(
    output_dir='./results',           # output directory where the model predictions and checkpoints will be written.
    learning_rate=2e-5,               # learning rate
    evaluation_strategy = "epoch",    # you can either define epoch or steps for your training cycles
    per_device_train_batch_size=16,   # batch size per GPU/TPU core/CPU for training
    per_device_eval_batch_size=16,    # batch size per GPU/TPU core/CPU for evaluation
    num_train_epochs=10,               # total number of training epochs
    weight_decay=0.01,  
    
                  # strength of weight decay (regularisation technique to reduce overfitting), higher values of weight_decay result in stronger regularization
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_emotions['train'],
    eval_dataset=tokenized_emotions['test'],
    tokenizer=tokenizer,
    data_collator=data_collator
)
trainer.train()

  5%|▌         | 500/10000 [00:30<08:07, 19.48it/s] 

{'loss': 0.7169, 'grad_norm': 3.7223079204559326, 'learning_rate': 1.9e-05, 'epoch': 0.5}


 10%|█         | 1000/10000 [00:58<08:11, 18.33it/s]

{'loss': 0.2543, 'grad_norm': 0.8517903089523315, 'learning_rate': 1.8e-05, 'epoch': 1.0}





























                                           
                                                    
 10%|█         | 1001/10000 [01:02<1:33:24,  1.61it/s]

...

100%|██████████| 10000/10000 [10:11<00:00, 18.78it/s]

{'loss': 0.0142, 'grad_norm': 0.003499669022858143, 'learning_rate': 0.0, 'epoch': 10.0}





























                                           
                                                     
100%|██████████| 10000/10000 [10:14<00:00, 18.78it/s]

100%|██████████| 10000/10000 [10:14<00:00, 16.26it/s]

{'eval_loss': 0.446134090423584, 'eval_runtime': 2.8087, 'eval_samples_per_second': 712.07, 'eval_steps_per_second': 44.504, 'epoch': 10.0}
{'train_runtime': 614.8484, 'train_samples_per_second': 260.227, 'train_steps_per_second': 16.264, 'train_loss': 0.10627782917022705, 'epoch': 10.0}


TrainOutput(global_step=10000, training_loss=0.10627782917022705, metrics={'train_runtime': 614.8484, 'train_samples_per_second': 260.227, 'train_steps_per_second': 16.264, 'total_flos': 1945612950228864.0, 'train_loss': 0.10627782917022705, 'epoch': 10.0})

In [74]:
from transformers import TextClassificationPipeline

myClassifier = TextClassificationPipeline(model=model, tokenizer=tokenizer)

myClassifier("i didnt feel humiliated")

[{'label': 'sadness', 'score': 0.9999728202819824}]

In [87]:
text = emotions['test'][20]['text']
print(text)

im not sure the feeling of loss will ever go away but it may dull to a sweet feeling of nostalgia at what i shared in this life with my dad and the luck i had to have a dad for years


In [88]:
myClassifier(text)

[{'label': 'sadness', 'score': 0.9999444484710693}]

In [90]:

emotions['test'][20]['label_text']

'sadness'